In [1]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import pymysql
import folium
from folium import plugins

In [3]:
conn = pymysql.connect(host='localhost', user='root', password='2544',
                      db='facilitydb', charset='utf8')
cur = conn.cursor()

### 쿼리문 1

In [4]:
sql = 'SELECT 대상시설명, 위도, 경도 FROM protect_tbl'
cur.execute(sql)

15

In [5]:
row = cur.fetchall()

Dataname = ['시설명', '위도', '경도']
protect_df = pd.DataFrame(row, columns=Dataname)
protect_df['시설종류'] = '노인장애인보호구역'
protect_df = protect_df[['시설명', '시설종류', '위도', '경도']]
protect_df.head()

,시설명,시설종류,위도,경도
0,강서노인종합복지관,노인장애인보호구역,37.558123,126.847552
1,등촌4종합복지과경로당,노인장애인보호구역,37.562259,126.842895
2,천사노인요양원,노인장애인보호구역,37.546692,126.833486
3,가양시니어스타워,노인장애인보호구역,37.556989,126.859332
4,곰달래어르신복지센터,노인장애인보호구역,37.529060,126.848898


### 쿼리문 2

In [6]:
sql = 'SELECT 시설명, 위도, 경도 FROM job_tbl'
cur.execute(sql)

11

In [7]:
row = cur.fetchall()

Dataname = ['시설명', '위도', '경도']
job_df = pd.DataFrame(row, columns=Dataname)
job_df['시설종류'] = '장애인직업재활시설'
job_df = job_df[['시설명', '시설종류', '위도', '경도']]
job_df.head()

,시설명,시설종류,위도,경도
0,열림일터,장애인직업재활시설,37.554930,126.854630
1,그라나다보호작업센터,장애인직업재활시설,37.568609,126.850703
2,기쁜우리보호작업장,장애인직업재활시설,37.565237,126.853576
3,강서구휴먼희망일터,장애인직업재활시설,37.566945,126.841394
4,구립강서구직업재활센터,장애인직업재활시설,37.567464,126.851714


### 쿼리문 3

In [8]:
sql = 'SELECT 시설명, 위도, 경도 FROM rehab_tbl'
cur.execute(sql)

17

In [9]:
row = cur.fetchall()

Dataname = ['시설명', '위도', '경도']
rehab_df = pd.DataFrame(row, columns=Dataname)
rehab_df['시설종류'] = '장애인지역사회재활시설'
rehab_df = rehab_df[['시설명', '시설종류', '위도', '경도']]
rehab_df.head()

,시설명,시설종류,위도,경도
0,늘푸른나무복지관,장애인지역사회재활시설,37.568609,126.850703
1,강서구수어통역센터,장애인지역사회재활시설,37.553257,126.820768
2,기쁜우리복지관부설주간보호시설,장애인지역사회재활시설,37.565237,126.853576
3,강서뇌성마비복지관,장애인지역사회재활시설,37.574191,126.812563
4,주간보호센터햇볕교실,장애인지역사회재활시설,37.571981,126.806080


### 쿼리문 4

In [10]:
sql = 'SELECT 시설명, 위도, 경도 FROM resident_tbl'
cur.execute(sql)

28

In [11]:
row = cur.fetchall()

Dataname = ['시설명', '위도', '경도']
resident_df = pd.DataFrame(row, columns=Dataname)
resident_df['시설종류'] = '장애인거주시설'
resident_df = resident_df[['시설명', '시설종류', '위도', '경도']]
resident_df.head()

,시설명,시설종류,위도,경도
0,교남소망의집,장애인거주시설,37.548814,126.852000
1,임마누엘화곡1공동체,장애인거주시설,37.535572,126.858702
2,임마누엘화곡공동체,장애인거주시설,37.530744,126.853751
3,임마누엘등촌1공동체,장애인거주시설,37.558319,126.853870
4,교남공동생활가정4호,장애인거주시설,37.541614,126.845911


### 4개의 dataframe 통합

In [12]:
facility_gs = pd.concat([protect_df, job_df, rehab_df, resident_df], ignore_index=True)
facility_gs

,시설명,시설종류,위도,경도
0,강서노인종합복지관,노인장애인보호구역,37.558123,126.847552
1,등촌4종합복지과경로당,노인장애인보호구역,37.562259,126.842895
2,천사노인요양원,노인장애인보호구역,37.546692,126.833486
3,가양시니어스타워,노인장애인보호구역,37.556989,126.859332
4,곰달래어르신복지센터,노인장애인보호구역,37.529060,126.848898
...,...,...,...,...
66,여호와닛시의집,장애인거주시설,37.564459,126.809921
67,H2빌,장애인거주시설,37.585787,126.815043
68,교남공동생활가정2호,장애인거주시설,37.540851,126.852350
69,행복이가득한집 2호,장애인거주시설,37.529284,126.840008


In [13]:
facility_gs.to_csv('facility_gs.csv', encoding="utf-8-sig")

In [16]:
location_data = facility_gs[['위도','경도']].values[:len(facility_gs)].tolist()
center =[facility_gs.위도.mean(),facility_gs.경도.mean()]

In [17]:
map = folium.Map(location=center,zoom_start=13,width='100%')
for i in range(len(facility_gs)):
    facility_name=facility_gs.시설명[i]
    facility_color='black'
    folium.Circle(location=location_data[i],radius=1,color=facility_color,fill=True,fill_opacity=1, tooltip=facility_name).add_to(map)

plugins.HeatMap(location_data,radius=20).add_to(map)

In [18]:
map

In [19]:
conn.commit()